## Starting with the Spotify API

In [20]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [21]:
username = 'britopereiraa'
client_id ='17a3ed14ab434b33b28f19f8bea3337d'
client_secret = '7f848f9d2a024b15be5fa776b4dc7a05'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [22]:
print(token)

BQBgzJeCu8XA8mV-Az4jax4xypHCIEWmVX2ceKM5fa9c9z4UpTEyENc84U1WVZrlRkMDOjAo15Rv3II4GF3-Y7B5Zugl0C5W--oDuOqTXLFYrnQoB_OhEH_Q0nAQVOh_pikE3DhqX7ewGP-bWzNfeU03mtw


#### Playlists used:
    
Created a big playlist that puts together 3 of my main playlists:
- 'Let the Chicken Chill'
- 'Indie Lusofono'
- 'Verdad'

spotify:playlist:7jIYTDhFlZoybdvGm3e6oh

- 'DISLIKED SONGS'
spotify:playlist:5zVBvIZZ4qCWDv71UZ108k

In [23]:
credentials = json.load(open('authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

In [24]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Liked playlist

In [6]:
playlist_index = 0

playlists = json.load(open('playlists_like_dislike.json'))

playlists

[{'uri': 'spotify:user:Test_1:playlist:7jIYTDhFlZoybdvGm3e6oh', 'like': True},
 {'uri': 'spotify:user:Test_2:playlist:4cth4hjHHeCQmOUsqdZFTU', 'like': False},
 {'uri': 'spotify:user:Test_1:playlist:37i9dQZF1CyVymzd2SIVxU', 'like': True},
 {'uri': 'spotify:user:Test_1:playlist:37i9dQZF1E9LjM9BhKuBcX', 'like': True},
 {'uri': 'spotify:user:Test_1:playlist:37i9dQZF1EjuSiYQC7nQLU', 'like': True},
 {'uri': 'spotify:user:Test_1:playlist:37i9dQZF1EthLPFZkbiec7', 'like': True}]

In [7]:
playlist_index = 0

playlists = json.load(open('playlists_like_dislike.json'))
playlist_uri = playlists[playlist_index]['uri']
like = playlists[playlist_index]['like']


In [8]:
playlist_uri

'spotify:user:Test_1:playlist:7jIYTDhFlZoybdvGm3e6oh'

In [10]:
playlist_uri

'spotify:user:Test_1:playlist:7jIYTDhFlZoybdvGm3e6oh'

In [11]:
playlist_uri.split(':')[4]

'7jIYTDhFlZoybdvGm3e6oh'

In [12]:
uri = playlist_uri    
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]
results = {'items':[]}

In [13]:
uri

'spotify:user:Test_1:playlist:7jIYTDhFlZoybdvGm3e6oh'

In [14]:
username

'Test_1'

In [15]:
playlist_id

'7jIYTDhFlZoybdvGm3e6oh'

In [43]:
def master_function(uri):
    uri = playlist_uri    
    username = uri.split(':')[2]
    playlist_id = uri.split(':')[4]
    results = {'items':[]}

    for n in range(0,6000,100):
        new = sp.user_playlist_tracks(username, playlist_id,  offset = n)
        results['items'] += new['items']

        playlist_tracks_data = results
        playlist_tracks_id = []
        playlist_tracks_titles = []
        playlist_tracks_artists = []

        for track in playlist_tracks_data['items']:
            if track['track']['id'] is not None:
                playlist_tracks_id.append(track['track']['id'])
                playlist_tracks_titles.append(track['track']['name'])
                for artist in track['track']['artists']:
                    artist_list = []
                    artist_list.append(artist['name'])
                playlist_tracks_artists.append(artist_list[0])
    
    df = pd.DataFrame([])
    for i in range(0, len(playlist_tracks_id)):
        features = sp.audio_features(playlist_tracks_id[i])
        features_df = pd.DataFrame(features)
        df = df.append(features_df)

    df['title'] = playlist_tracks_titles
    #features_df['first_artist'] = playlist_tracks_first_artists
    df['main_artist'] = playlist_tracks_artists
    #features_df = features_df.set_index('id')
    df = df[['id', 'title', 'main_artist',
                               'danceability', 'energy', 'key', 'loudness',
                               'mode', 'acousticness', 'instrumentalness',
                               'liveness', 'valence', 'tempo',
                               'duration_ms', 'time_signature']]

    return df

In [9]:
liked_df = master_function(playlist_uri)
liked_df['Like'] = 1
liked_df.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Like
0,0TLAptKgYxe7F0KewWH6X6,I Need A Forest Fire,Bon Iver,0.293,0.290,2,-11.596,1,0.696,0.875000,0.1000,0.0373,128.276,257220,4,1
0,2y0rHDTckWQ7of6tI1yc82,Naked Life,Oko Ebombo,0.624,0.321,0,-9.776,1,0.463,0.056000,0.1030,0.3040,73.996,255030,4,1
0,5nVE45ithgyVocZp5XLAk6,The Bug Collector,Haley Heynderickx,0.538,0.419,10,-9.705,1,0.777,0.329000,0.0967,0.5290,99.622,228460,4,1
0,66VuLX5s1nrZzzQqJqPbri,Simply Are,Arto Lindsay,0.682,0.694,7,-11.393,0,0.289,0.714000,0.0961,0.9540,173.851,224107,4,1
0,0X61Ud5Uzv44RvY3s1SsuO,Imogen,Nick Mulvey,0.493,0.584,5,-6.894,1,0.850,0.000353,0.1140,0.2340,86.909,255533,4,1


In [10]:
liked_df.shape

(1327, 16)

### Disliked playlist

In [11]:
playlist_index = 1

playlists = json.load(open('playlists_like_dislike.json'))
playlist_uri = playlists[playlist_index]['uri']
dislike = playlists[playlist_index]['like']

#Getting Information of tracks in Spotify playlist

uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]

results = sp.user_playlist_tracks(username, playlist_id,  offset = 0)

In [12]:
playlist_id

'4cth4hjHHeCQmOUsqdZFTU'

In [13]:
disliked_df = master_function(playlist_uri)
disliked_df['Like'] = 0
disliked_df.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Like
0,2VMMhJxWI2aipSBTkpqXjI,BPM 170,MC Kevin o Chris,0.710,0.504,6,-5.212,1,0.605,0.00321,0.0880,0.895,84.318,203520,4,0
0,0K6ldKbWkd91UJS3MlTRyR,Largadão,Lexa,0.888,0.920,11,-1.673,1,0.358,0.00182,0.0666,0.764,93.987,132766,4,0
0,25gD4PQjxCLnwIY2n23aEl,Flores,Luísa Sonza,0.626,0.599,6,-5.869,0,0.255,0.00000,0.1600,0.621,100.004,155798,4,0
0,4eqpOs8YgVHLJPnBPD41zN,Mete Dança,Ruxell,0.754,0.928,1,-2.120,1,0.281,0.00008,0.1440,0.818,99.746,122400,4,0
0,7jV67Bz9qRkk57FMedir8S,Nosso Relacionamento,Dallass,0.568,0.471,8,-8.769,0,0.443,0.00000,0.0929,0.455,134.817,200769,4,0


### Final dataframe

In [14]:
frames = [liked_df, disliked_df]

my_data = pd.concat(frames)

In [15]:
my_data

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Like
0,0TLAptKgYxe7F0KewWH6X6,I Need A Forest Fire,Bon Iver,0.293,0.290,2,-11.596,1,0.69600,0.875000,0.1000,0.0373,128.276,257220,4,1
0,2y0rHDTckWQ7of6tI1yc82,Naked Life,Oko Ebombo,0.624,0.321,0,-9.776,1,0.46300,0.056000,0.1030,0.3040,73.996,255030,4,1
0,5nVE45ithgyVocZp5XLAk6,The Bug Collector,Haley Heynderickx,0.538,0.419,10,-9.705,1,0.77700,0.329000,0.0967,0.5290,99.622,228460,4,1
0,66VuLX5s1nrZzzQqJqPbri,Simply Are,Arto Lindsay,0.682,0.694,7,-11.393,0,0.28900,0.714000,0.0961,0.9540,173.851,224107,4,1
0,0X61Ud5Uzv44RvY3s1SsuO,Imogen,Nick Mulvey,0.493,0.584,5,-6.894,1,0.85000,0.000353,0.1140,0.2340,86.909,255533,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3485UyMhs2ZF50l9sIZEys,FINAIS,benji price,0.750,0.615,11,-7.532,0,0.64200,0.000000,0.2380,0.8330,75.986,207749,4,0
0,7szuecWAPwGoV1e5vGu8tl,In Your Eyes,The Weeknd,0.667,0.719,7,-5.371,0,0.00285,0.000081,0.0736,0.7170,100.021,237520,4,0
0,4Ty4XfOHdUFIXmVtBIFMIG,XPIDI,benji price,0.840,0.475,5,-6.665,0,0.64100,0.000000,0.0983,0.8820,77.006,172064,4,0
0,1xQ6trAsedVPCdbtDAmk0c,Savage Love (Laxed - Siren Beat),Jason Derulo,0.767,0.481,0,-8.520,0,0.23400,0.000000,0.2690,0.7610,150.076,171375,4,0


In [16]:
#export csv

my_data.to_csv (r'../data/my_data.csv', index = False, header=True)

## My Musical Journey 

In [17]:
#2016
playlist_index = 2

playlists = json.load(open('playlists_like_dislike.json'))
playlist_uri = playlists[playlist_index]['uri']
dislike = playlists[playlist_index]['like']

#Getting Information of tracks in Spotify playlist

uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]

results = sp.user_playlist_tracks(username, playlist_id,  offset = 0)

In [18]:
df_2016 = master_function(playlist_uri)
df_2016['Year'] = 2016
df_2016.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Year
0,7gARZ2i5u5DHC1b1n1fML5,Breezeblocks,alt-J,0.569,0.645,5,-7.368,1,0.1180,0.001140,0.1460,0.247,74.985,227080,4,2016
0,0LPNhjjjg432PrqsDUcLjs,Sleep by Rivers,Kyle Lionhart,0.639,0.361,9,-13.296,1,0.6700,0.000333,0.1210,0.407,86.530,241023,4,2016
0,1xCkZYGKDv7ELjSlIKedOy,No Diggity,Chet Faker,0.672,0.506,2,-12.013,0,0.5890,0.000024,0.0925,0.500,79.974,225000,4,2016
0,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Rik Rok,0.853,0.606,0,-4.596,1,0.0561,0.000000,0.3130,0.654,94.759,227600,4,2016
0,7iL6o9tox1zgHpKUfh9vuC,In Da Club,50 Cent,0.899,0.713,6,-2.752,0,0.2550,0.000000,0.0708,0.777,90.051,193467,4,2016


In [19]:
#2017
playlist_index = 3

playlists = json.load(open('playlists_like_dislike.json'))
playlist_uri = playlists[playlist_index]['uri']
dislike = playlists[playlist_index]['like']

#Getting Information of tracks in Spotify playlist

uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]

results = sp.user_playlist_tracks(username, playlist_id,  offset = 0)

df_2017 = master_function(playlist_uri)
df_2017['Year'] = 2017
df_2017.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Year
0,7gARZ2i5u5DHC1b1n1fML5,Breezeblocks,alt-J,0.569,0.645,5,-7.368,1,0.118,0.001140,0.1460,0.247,74.985,227080,4,2017
0,36i9NcqbdQFdQfOROpIwMA,Fico Assim Sem Você,Adriana Partimpim,0.809,0.428,10,-8.428,1,0.809,0.000000,0.0851,0.507,121.969,188147,4,2017
0,36ylLzMtvJ2CItc7bN4TcF,Taro,alt-J,0.350,0.476,5,-10.653,1,0.765,0.003900,0.0785,0.293,171.938,314987,4,2017
0,1NyFRrFiJAJIaR6icj1goI,Skinny Love,Bon Iver,0.592,0.256,4,-14.031,0,0.842,0.000002,0.0880,0.103,76.361,238533,4,2017
0,6O1XKTvgVLwytNWdOlFcHI,3WW,alt-J,0.637,0.137,2,-15.131,1,0.740,0.028700,0.0655,0.146,78.017,300107,4,2017


In [20]:
df_2017

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Year
0,7gARZ2i5u5DHC1b1n1fML5,Breezeblocks,alt-J,0.569,0.645,5,-7.368,1,0.1180,0.001140,0.1460,0.2470,74.985,227080,4,2017
0,36i9NcqbdQFdQfOROpIwMA,Fico Assim Sem Você,Adriana Partimpim,0.809,0.428,10,-8.428,1,0.8090,0.000000,0.0851,0.5070,121.969,188147,4,2017
0,36ylLzMtvJ2CItc7bN4TcF,Taro,alt-J,0.350,0.476,5,-10.653,1,0.7650,0.003900,0.0785,0.2930,171.938,314987,4,2017
0,1NyFRrFiJAJIaR6icj1goI,Skinny Love,Bon Iver,0.592,0.256,4,-14.031,0,0.8420,0.000002,0.0880,0.1030,76.361,238533,4,2017
0,6O1XKTvgVLwytNWdOlFcHI,3WW,alt-J,0.637,0.137,2,-15.131,1,0.7400,0.028700,0.0655,0.1460,78.017,300107,4,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0o21KBZdbK4M0iY3mBwVmm,Trevo (Tu),Diogo Piçarra,0.599,0.461,10,-7.998,1,0.7760,0.000000,0.0969,0.4670,99.766,206521,4,2017
0,1DI96QeWKpB5h8gayx4Cqn,Clean The House,Fat Freddy's Drop,0.794,0.632,9,-6.305,0,0.0638,0.457000,0.1360,0.2280,81.004,254112,4,2017
0,6IfgqeI5iCAk105wZZwt1p,Quebramos os Dois,Toranja,0.389,0.203,7,-12.968,1,0.9000,0.000283,0.0592,0.0372,97.763,255640,1,2017
0,3z7quP9A9iNsTiwGeutzV6,Fado Toninho,Deolinda,0.692,0.210,0,-9.525,1,0.8650,0.000000,0.1600,0.7430,77.878,127400,4,2017


In [21]:
#2018
playlist_index = 4

playlists = json.load(open('playlists_like_dislike.json'))
playlist_uri = playlists[playlist_index]['uri']
dislike = playlists[playlist_index]['like']

#Getting Information of tracks in Spotify playlist

uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]

results = sp.user_playlist_tracks(username, playlist_id,  offset = 0)

df_2018 = master_function(playlist_uri)
df_2018['Year'] = 2018
df_2018.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Year
0,6RBAJlxHY9Ulvw2y7yk4aX,Forrest Gump,Frank Ocean,0.701,0.417,6,-7.818,0,0.0888,0.000648,0.3750,0.186,95.003,194840,4,2018
0,0tCcebB7qaXD0QnwLMoYyN,1995-12-16,Sam The Kid,0.543,0.829,3,-6.147,0,0.3360,0.000000,0.3260,0.712,98.295,474044,4,2018
0,6t1ahr2f1t9iSjjNFwFb4C,Jungle,Tash Sultana,0.589,0.699,1,-5.883,0,0.2850,0.000081,0.0881,0.496,145.997,315603,4,2018
0,6bWBk9yZLG6xHgRgEtMqhZ,Sina,Banda Maré,0.620,0.588,1,-8.086,1,0.7350,0.000000,0.1200,0.571,76.921,170201,4,2018
0,2x5Frs6ts14U4MSLrQQlXg,Why iii Love The Moon.,Phony Ppl,0.706,0.483,1,-7.929,1,0.5470,0.000002,0.1360,0.663,92.002,331827,4,2018


In [22]:
#2019
playlist_index = 5

playlists = json.load(open('playlists_like_dislike.json'))
playlist_uri = playlists[playlist_index]['uri']
dislike = playlists[playlist_index]['like']

#Getting Information of tracks in Spotify playlist

uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]

results = sp.user_playlist_tracks(username, playlist_id,  offset = 0)

df_2019 = master_function(playlist_uri)
df_2019['Year'] = 2019
df_2019.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Year
0,6t1ahr2f1t9iSjjNFwFb4C,Jungle,Tash Sultana,0.589,0.699,1,-5.883,0,0.2850,0.000081,0.0881,0.496,145.997,315603,4,2019
0,7mptueK99zxqmgIdT3SaWF,Spirit Bird,Xavier Rudd,0.392,0.315,0,-10.371,1,0.4880,0.000000,0.1270,0.253,126.243,429236,4,2019
0,1B0BQaSRHxhI0AUlItY3LK,MALAMENTE - Cap.1: Augurio,ROSALÍA,0.893,0.482,0,-4.779,0,0.5490,0.016400,0.0752,0.530,97.975,150027,4,2019
0,0wXuerDYiBnERgIpbb3JBR,Redbone,Childish Gambino,0.743,0.347,1,-11.174,1,0.1670,0.009510,0.1030,0.572,160.143,326933,4,2019
0,0NnVzBW6TJLaEzcyXjqRDH,Mystik,Tash Sultana,0.548,0.699,2,-7.149,0,0.0223,0.000392,0.0903,0.308,132.798,249296,4,2019


In [23]:
#merge

frames = [df_2016,df_2017, df_2018,df_2019]

top_year = pd.concat(frames)

In [24]:
#export csv

top_year.to_csv (r'../data/top_year.csv', index = False, header=True)

### SUPER DATABASE WITH A LOT OF MUSICS

In [44]:
#SUPER BIG PLAYLIST
playlist_index = 6

playlists = json.load(open('playlists_like_dislike.json'))
playlist_uri = playlists[playlist_index]['uri']
dislike = playlists[playlist_index]['like']

#Getting Information of tracks in Spotify playlist

uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]

results = sp.user_playlist_tracks(username, playlist_id,  offset = 0)

all_songs = master_function(playlist_uri)
all_songs.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2aQ8NyRJcsQFgb8jwUil90,Que Bandeira,Evinha,0.755,0.537,1,-10.679,1,0.75600,0.000003,0.3550,0.811,93.393,149420,4
0,6d4KqeYESkFHMirUgE1Aa3,Não Vá,Tim Maia,0.658,0.741,2,-10.495,1,0.04430,0.001280,0.0603,0.928,103.283,307037,4
0,2OoOgl80tDTElnbMYfhJyH,Reggins,The Blackbyrds,0.645,0.383,5,-16.549,0,0.00495,0.611000,0.0727,0.943,97.238,244800,4
0,5voWjUklIhjnNQU5aZCIm4,Dreaming About You,The Blackbyrds,0.744,0.292,4,-16.592,0,0.26500,0.095900,0.0738,0.881,96.366,352400,4
0,2jHa1UWWuouaoFqE8I7W77,Never Let Her Go,David Gates,0.576,0.399,8,-14.764,1,0.23300,0.000022,0.1040,0.621,84.049,188813,4


In [46]:
all_songs.shape #woooow so big!! 

(5255, 15)

In [47]:
#export csv

all_songs.to_csv (r'../data/all_songs.csv', index = False, header=True)